<h1>SIT307 Group Assignment 1 notebook</h1>

<h3>Group 3</h3>
<p>By:<br>
    Aaron Norwood,218330434<br>
    Joshua Anthony, 219466473<br>
    Roger Middenway, 217602784<br>
    David Adams, 216110104<br>
    Linden Hutchinson, 218384326<br>
    Dale Orders, 219106283

# Imported libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
from scipy import stats
import plotly.express as px
%matplotlib inline

<h3> Read in the data, store if dataframe

In [2]:
df = pd.read_csv('./data/healthcare-dataset-stroke-data.csv')

<h4>Tidying

In [3]:
##convert gender to lowercase
df['gender'] = df['gender'].apply(lambda x: x.lower())

# print(len(df[df['gender'] == 'other']))
## drop the one row that has "other" gender to keep things simple
df.drop(index=df[df['gender'] == 'other'].index, inplace=True)

# replace gender with binary values
df['gender'] = df['gender'].str.lower().map({'male': 1, 'female': 0})

# replace marital status with binary values
df['ever_married'] = df['ever_married'].str.lower().map({'yes': 1, 'no': 0})

##convert work_type to lowercase ensure consistent spacing 
df['work_type'] = df['work_type'].apply(lambda x: x.lower().replace('_','-'))

##convert residence_type to lowercase
df.rename(columns={'Residence_type':'residence_type'}, inplace=True)
df['residence_type'] = df['residence_type'].apply(lambda x: x.lower())

##convert smoking_status to lowercase ensure consistent spacing 
df['smoking_status'] = df['smoking_status'].apply(lambda x: x.lower().replace(' ', '-'))

## round off age
df['age'] = df['age'].apply(lambda x : round(x))

# cap BMI outliers to a maximum of 60 
df['bmi'] = df['bmi'].apply(lambda bmi_value: bmi_value if 12 < bmi_value < 60 else np.nan)

<h4>Imputing

In [4]:
m_df = df[df['gender'] == 1]
f_df = df[df['gender'] == 0]

m_bmi_avg = m_df.groupby('age')['bmi'].mean()
f_bmi_avg = f_df.groupby('age')['bmi'].mean()
##round to one to fit with other bmi values
m_bmi_avg = round(m_bmi_avg,1)
f_bmi_avg = round(f_bmi_avg,1)

missing_vals = df[df.isnull().any(axis = 1)]

for index, row in missing_vals.iterrows():
    if row['gender'] == 1:
        df.loc[index,['bmi']] = m_bmi_avg[row['age']]
    else:
        df.loc[index,['bmi']] = f_bmi_avg[row['age']]

<h4>Binning

In [5]:
# implement age bins
df['age_bin'] = pd.qcut(df['age'], q = 10, precision=1)

# implement glucose bins
df['avg_glucose_level_bin'] = pd.qcut(df['avg_glucose_level'], q=10, precision=1)

# implement bmi bins
df['bmi_bin'] = pd.qcut(df['bmi'], q=10, precision=1)

<h4>Mapping

In [6]:
work_type_map = {
    'private':0,
    'self-employed':1,
    'govt-job':2,
    'children':3,
    'never-worked':4
}
residence_type_map = {
    'urban':0,
    'rural':1
}
smoking_status_map = {
    'formerly-smoked':0,
    'never-smoked':1,
    'smokes':2,
    'unknown':3
}

df['work_type'] = df['work_type'].map(work_type_map)
df['residence_type'] = df['residence_type'].map(residence_type_map)
df['smoking_status'] = df['smoking_status'].map(smoking_status_map)

In [7]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_bin,avg_glucose_level_bin,bmi_bin
0,9046,1,67,0,1,1,0,0,228.69,36.6,0,1,"(65.0, 75.0]","(192.2, 271.7]","(34.2, 38.6]"
1,51676,0,61,0,0,1,1,1,202.21,29.1,1,1,"(57.0, 65.0]","(192.2, 271.7]","(28.2, 29.9]"
2,31112,1,80,0,1,1,0,1,105.92,32.5,1,1,"(75.0, 82.0]","(98.9, 108.5]","(31.8, 34.2]"
3,60182,0,49,0,0,1,0,0,171.23,34.4,2,1,"(45.0, 51.0]","(124.1, 192.2]","(34.2, 38.6]"
4,1665,0,79,1,0,1,1,1,174.12,24.0,1,1,"(75.0, 82.0]","(124.1, 192.2]","(22.6, 24.7]"


In [8]:
import numpy as np 
import pandas as pd
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
import seaborn as sns; sns.set()

from sklearn import tree

import numpy as np 
import pandas as pd 
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from imblearn.over_sampling import SMOTE


import warnings
warnings.filterwarnings("ignore")

features = ['age', 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi']

label = ['stroke']

X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101) 
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)


sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.linear_model import LogisticRegression

LR_model = LogisticRegression(max_iter=20000)

# fit the model
LR_model.fit(X_train, y_train)

y_predict_LR = LR_model.predict(X_test)

# model score
predict_train_LR = LR_model.predict(X_train)
predict_test_LR = LR_model.predict(X_test)

# accuracy score
LR_train_score = LR_model.score(X_train,y_train)
LR_test_score = LR_model.score(X_test,y_test)

# f1-score
LR_f1_score = metrics.f1_score(y_test,predict_test_LR)
LR_recall = metrics.recall_score(y_test,predict_test_LR)

print('Accuracy on Train set',LR_train_score)
print('Accuracy on Test set',LR_test_score)
print('F1-score on Test set:',LR_f1_score)
print(metrics.classification_report(y_test, predict_test_LR))


 


Accuracy on Train set 0.9541338007595677
Accuracy on Test set 0.9489916963226572
F1-score on Test set: 0.0
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       800
           1       0.00      0.00      0.00        43

    accuracy                           0.95       843
   macro avg       0.47      0.50      0.49       843
weighted avg       0.90      0.95      0.92       843



In [9]:

features = ['age',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi']

label = ['stroke']

X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101) 
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

LR_model = DecisionTreeClassifier()

# fit the model
LR_model.fit(X_train, y_train)

y_predict_LR = LR_model.predict(X_test)

# model score
predict_train_LR = LR_model.predict(X_train)
predict_test_LR = LR_model.predict(X_test)

# accuracy score
LR_train_score = LR_model.score(X_train,y_train)
LR_test_score = LR_model.score(X_test,y_test)

# f1-score
LR_f1_score = metrics.f1_score(y_test,predict_test_LR)
LR_recall = metrics.recall_score(y_test,predict_test_LR)

print('Accuracy on Train set',LR_train_score)
print('Accuracy on Test set',LR_test_score)
print('F1-score on Test set:',LR_f1_score)
print(metrics.classification_report(y_test, predict_test_LR))




Accuracy on Train set 1.0
Accuracy on Test set 0.9039145907473309
F1-score on Test set: 0.0689655172413793
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       800
           1       0.07      0.07      0.07        43

    accuracy                           0.90       843
   macro avg       0.51      0.51      0.51       843
weighted avg       0.90      0.90      0.90       843



In [14]:
import warnings
from imblearn.over_sampling import SMOTE, RandomOverSampler
warnings.filterwarnings("ignore")

features = ['age',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi']

label = ['stroke']

X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101) 
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
X_train_res, y_train_res = SMOTE().fit_resample(X_train, y_train)

sc=StandardScaler()

X_train = sc.fit_transform(X_train_res)
X_test = sc.transform(X_test)

LR_model = KNeighborsClassifier()

# fit the model
LR_model.fit(X_train_res, y_train_res)

y_predict_LR = LR_model.predict(X_test)

# model score
predict_train_LR = LR_model.predict(X_train_res)
predict_test_LR = LR_model.predict(X_test)

# accuracy score
LR_train_score = LR_model.score(X_train_res,y_train_res)
LR_test_score = LR_model.score(X_test,y_test)

# f1-score
LR_f1_score = metrics.f1_score(y_test,predict_test_LR)
LR_recall = metrics.recall_score(y_test,predict_test_LR)

print('Accuracy on Train set',LR_train_score)
print('Accuracy on Test set',LR_test_score)
print('F1-score on Test set:',LR_f1_score)
print(metrics.classification_report(y_test, predict_test_LR))



Accuracy on Train set 0.9327924066135946
Accuracy on Test set 0.9489916963226572
F1-score on Test set: 0.0
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       800
           1       0.00      0.00      0.00        43

    accuracy                           0.95       843
   macro avg       0.47      0.50      0.49       843
weighted avg       0.90      0.95      0.92       843



In [25]:
import warnings
from imblearn.over_sampling import SMOTE, RandomOverSampler
warnings.filterwarnings("ignore")

features = ['age',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi']

label = ['stroke']

X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101) 
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
X_train_res, y_train_res = SMOTE().fit_resample(X_train, y_train)

sc=StandardScaler()

X_train = sc.fit_transform(X_train_res)
X_test = sc.transform(X_test)

LR_model = GradientBoostingClassifier()

# fit the model
LR_model.fit(X_train_res, y_train_res)

y_predict_LR = LR_model.predict(X_test)

# model score
predict_train_LR = LR_model.predict(X_train_res)
predict_test_LR = LR_model.predict(X_test)

# accuracy score
LR_train_score = LR_model.score(X_train_res,y_train_res)
LR_test_score = LR_model.score(X_test,y_test)

# f1-score
LR_f1_score = metrics.f1_score(y_test,predict_test_LR)
LR_recall = metrics.recall_score(y_test,predict_test_LR)

print('Accuracy on Train set',LR_train_score)
print('Accuracy on Test set',LR_test_score)
print('F1-score on Test set:',LR_f1_score)
print(metrics.classification_report(y_test, predict_test_LR))


Accuracy on Train set 0.8733925290875689
Accuracy on Test set 0.9489916963226572
F1-score on Test set: 0.0
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       800
           1       0.00      0.00      0.00        43

    accuracy                           0.95       843
   macro avg       0.47      0.50      0.49       843
weighted avg       0.90      0.95      0.92       843

